In [1]:
!wget -q https://git.io/J0fjL -O IAM_Words.zip
!unzip -qq IAM_Words.zip

[IAM_Words.zip]
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of IAM_Words.zip or
        IAM_Words.zip.zip, and cannot find IAM_Words.zip.ZIP, period.


In [2]:
!mkdir data
!mkdir data/words
!tar -xf IAM_Words/words.tgz -C data/words
!mv IAM_Words/words.txt data/words

tar: IAM_Words/words.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now
mv: cannot stat 'IAM_Words/words.txt': No such file or directory


In [3]:
!unzip drive/MyDrive/Dataset/lines.zip -d data/lines

Výstup streamovania bol skrátený na tento počet posledných riadkov: 5000.
  inflating: data/lines/lines/img/h07/h07-028/h07-028-10.png  
   creating: data/lines/lines/img/h07/h07-028a/
  inflating: data/lines/lines/img/h07/h07-028a/h07-028a-00.png  
  inflating: data/lines/lines/img/h07/h07-028a/h07-028a-01.png  
  inflating: data/lines/lines/img/h07/h07-028a/h07-028a-02.png  
  inflating: data/lines/lines/img/h07/h07-028a/h07-028a-03.png  
  inflating: data/lines/lines/img/h07/h07-028a/h07-028a-04.png  
  inflating: data/lines/lines/img/h07/h07-028a/h07-028a-05.png  
  inflating: data/lines/lines/img/h07/h07-028a/h07-028a-06.png  
  inflating: data/lines/lines/img/h07/h07-028a/h07-028a-07.png  
 extracting: data/lines/lines/img/h07/h07-028a/h07-028a-08.png  
   creating: data/lines/lines/img/h07/h07-030/
  inflating: data/lines/lines/img/h07/h07-030/h07-030-00.png  
  inflating: data/lines/lines/img/h07/h07-030/h07-030-01.png  
  inflating: data/lines/lines/img/h07/h07-030/h07-030-02.

In [4]:
!pip install editdistance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
 import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os
from pathlib import Path
from tqdm import tqdm
from tensorflow.keras.layers.experimental.preprocessing import StringLookup
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer, Conv2D, Multiply, Activation

from tensorflow.keras.layers import Conv2D, Bidirectional, LSTM, GRU, Dense
from tensorflow.keras.layers import Dropout, BatchNormalization, LeakyReLU, PReLU
from tensorflow.keras.layers import Input, Add, Activation, Lambda, MaxPooling2D, Reshape

from tensorflow import keras

import editdistance

AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#     try:
#         # Currently, memory growth needs to be the same across GPUs
#         for gpu in gpus:
#             tf.config.experimental.set_memory_growth(gpu, True)
#         logical_gpus = tf.config.list_logical_devices('GPU')
#         print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#     except RuntimeError as e:
#         # Memory growth must be set before GPUs have been initialized
#         print(e)

Physical devices cannot be modified after being initialized


In [7]:
characters = " |!\"#%&'()*+,-./0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz´ÁÉÍÓÔÚÝáéíúýĆČčĎĽľŇňŠšŤťŽž€"

exception_img = ["data/words/a01/a01-117/a01-117-05-02.png", "data/words/a01/a01-117/a01-117-05-02.png", "data/words/r06/r06-022/r06-022-03-05.png", ]

samples = []
max_len = 60
def update_samples(labels_path, img_root_path):
  global samples
  with open(labels_path, 'r', encoding="utf-8") as f:
      for idx, line in tqdm(enumerate(f)):
          # if idx > 2000:
          #   break
          if line[0] == '#':
              continue
          line = line.split(' ')
          if line[1] != 'ok':
              continue

          name = line[0].split('-')
          img_path = "/".join([img_root_path, name[0], f"{name[0]}-{name[1]}", line[0]]) + ".png"
          gt_text = " ".join(line[8:]).strip()

          gt_text = gt_text.replace("|", " ")

          if len(gt_text) >= max_len:
              continue

          if not os.path.getsize(img_path) or len(gt_text)==0:
            print(img_path)
            continue

          samples.append((img_path, gt_text))


update_samples("data/lines/lines/gt/lines.txt", "data/lines/lines/img")
# update_samples("data/words/words.txt", "data/words")
np.random.shuffle(samples)

split_idx = int(len(samples)*0.9)
samples_trian = samples[:split_idx]
samples_validation = samples[split_idx:]
characters = sorted(list(characters))

print("Samples", len(samples))

13397it [00:00, 117922.16it/s]

Samples 10860


In [8]:
batch_size = 32
image_width = 512
image_height = 64
padding_token = 120

char_to_num = StringLookup(vocabulary=characters)
num_to_char = StringLookup(vocabulary=char_to_num.get_vocabulary(), invert=True)

def distortion_free_resize(image, img_size):
    w, h = img_size
    image = tf.image.resize(image, size=(h, w), preserve_aspect_ratio=True)

    # Check tha amount of padding needed to be done.
    pad_height = h - tf.shape(image)[0]
    pad_width = w - tf.shape(image)[1]

    # Only necessary if you want to do same amount of padding on both sides.
    if pad_height % 2 != 0:
        height = pad_height // 2
        pad_height_top = height + 1
        pad_height_bottom = height
    else:
        pad_height_top = pad_height_bottom = pad_height // 2

    if pad_width % 2 != 0:
        width = pad_width // 2
        pad_width_left = width + 1
        pad_width_right = width
    else:
        pad_width_left = pad_width_right = pad_width // 2

    image = tf.pad(
        image,
        paddings=[
            [pad_height_top, pad_height_bottom],
            [pad_width_left, pad_width_right],
            [0, 0],
        ],
    )

    image = tf.transpose(image, perm=[1, 0, 2])
    image = tf.image.flip_left_right(image)
    return image

def preprocess_image(image_path, img_size=(image_width, image_height)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, 1)
    image = distortion_free_resize(image, img_size)
    image = tf.cast(image, tf.float32) / 255.0
    return image

def vectorize_label(label):
    label = char_to_num(tf.strings.unicode_split(label, input_encoding="UTF-8"))
    length = tf.shape(label)[0]
    pad_amount = max_len - length
    label = tf.pad(label, paddings=[[0, pad_amount]], constant_values=padding_token)
    return label

tf.function
def process_images_labels(sample):
    image = preprocess_image(sample[0])
    label = vectorize_label(sample[1])
    
    # return image, label
    return {"image": image, "label": label}


def prepare_dataset(samples):
    dataset = tf.data.Dataset.from_tensor_slices(samples).map(
        process_images_labels, num_parallel_calls=AUTOTUNE
    )
    return dataset.padded_batch(batch_size).cache().prefetch(AUTOTUNE)

train_ds = prepare_dataset(samples_trian)
validation_ds = prepare_dataset(samples_validation)

In [9]:
class CTCLayer(tf.keras.layers.Layer):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.loss_fn = tf.keras.backend.ctc_batch_cost

    def call(self, y_true, y_pred):
        batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
        input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
        label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

        input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
        label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")
        loss = self.loss_fn(y_true, y_pred, input_length, label_length)
        # print(loss)
        
        loss = tf.math.reduce_mean(loss)
        
        
        self.add_loss(loss)

        # At test time, just return the computed predictions.
        return y_pred

In [10]:
"""
Tensorflow Keras layer implementation of the octave convolution.

Reference (based):
    Yunpeng Chen, Haoqi Fan, Bing Xu, Zhicheng Yan, Yannis Kalantidis, Marcus Rohrbach, Shuicheng Yan, Jiashi Feng.
    Drop an Octave: Reducing Spatial Redundancy in Convolutional Neural Networks with Octave Convolution.

    OctConv-TFKeras
    Github: https://github.com/koshian2/OctConv-TFKeras
"""


class OctConv2D(Layer):
    """Octave Convolutional Class"""

    def __init__(self,
                 filters,
                 alpha,
                 kernel_size=(3,3),
                 strides=(1,1),
                 padding="same",
                 kernel_initializer="glorot_uniform",
                 kernel_regularizer=None,
                 kernel_constraint=None,
                 **kwargs):
        assert alpha >= 0 and alpha <= 1
        assert filters > 0 and isinstance(filters, int)

        super().__init__(**kwargs)

        self.alpha = alpha
        self.filters = filters
        # optional values
        self.kernel_size = kernel_size
        self.strides = strides
        self.padding = padding
        self.kernel_initializer = kernel_initializer
        self.kernel_regularizer = kernel_regularizer
        self.kernel_constraint = kernel_constraint

        # --> low channels
        self.low_channels = int(self.filters * self.alpha)
        # --> high channels
        self.high_channels = self.filters - self.low_channels

    def build(self, input_shape):
        assert len(input_shape) == 2
        assert len(input_shape[0]) == 4 and len(input_shape[1]) == 4
        # assertion for high inputs
        assert input_shape[0][1] // 2 >= self.kernel_size[0]
        assert input_shape[0][2] // 2 >= self.kernel_size[1]
        # assertion for low inputs
        assert input_shape[0][1] // input_shape[1][1] == 2
        assert input_shape[0][2] // input_shape[1][2] == 2

        assert K.image_data_format() == "channels_last"
        high_in = int(input_shape[0][3])
        low_in = int(input_shape[1][3])

        # High -> Low
        self.high_to_high_kernel = self.add_weight(name="high_to_high_kernel",
                                                   shape=(*self.kernel_size, high_in, self.high_channels),
                                                   initializer=self.kernel_initializer,
                                                   regularizer=self.kernel_regularizer,
                                                   constraint=self.kernel_constraint)
        # High -> Low
        self.high_to_low_kernel = self.add_weight(name="high_to_low_kernel",
                                                  shape=(*self.kernel_size, high_in, self.low_channels),
                                                  initializer=self.kernel_initializer,
                                                  regularizer=self.kernel_regularizer,
                                                  constraint=self.kernel_constraint)

        # Low -> High
        self.low_to_high_kernel = self.add_weight(name="low_to_high_kernel",
                                                  shape=(*self.kernel_size, low_in, self.high_channels),
                                                  initializer=self.kernel_initializer,
                                                  regularizer=self.kernel_regularizer,
                                                  constraint=self.kernel_constraint)
        # Low -> Low
        self.low_to_low_kernel = self.add_weight(name="low_to_low_kernel",
                                                 shape=(*self.kernel_size, low_in, self.low_channels),
                                                 initializer=self.kernel_initializer,
                                                 regularizer=self.kernel_regularizer,
                                                 constraint=self.kernel_constraint)
        super().build(input_shape)

    def call(self, inputs):
        # Input=[x^H, x^L]
        assert len(inputs) == 2
        high_input, low_input = inputs
        # High -> High conv
        high_to_high = K.conv2d(high_input, self.high_to_high_kernel,
                                strides=self.strides, padding=self.padding,
                                data_format="channels_last")
        # High -> low conv
        high_to_low = K.pool2d(high_input, (2, 2), strides=(2, 2), pool_mode="avg")
        high_to_low = K.conv2d(high_to_low, self.high_to_low_kernel, strides=self.strides,
                               padding=self.padding, data_format="channels_last")

        # Low -> high conv
        low_to_high = K.conv2d(low_input, self.low_to_high_kernel,
                               strides=self.strides, padding=self.padding,
                               data_format="channels_last")
        low_to_high = K.repeat_elements(low_to_high, 2, axis=1)
        low_to_high = K.repeat_elements(low_to_high, 2, axis=2)

        # Low -> low conv
        low_to_low = K.conv2d(low_input, self.low_to_low_kernel,
                              strides=self.strides, padding=self.padding,
                              data_format="channels_last")

        # cross add
        high_add = high_to_high + low_to_high
        low_add = low_to_low + high_to_low

        return [high_add, low_add]

    def compute_output_shape(self, input_shapes):
        high_in_shape, low_in_shape = input_shapes
        high_out_shape = (*high_in_shape[:3], self.high_channels)
        low_out_shape = (*low_in_shape[:3], self.low_channels)
        return [high_out_shape, low_out_shape]

    def get_config(self):
        base_config = super().get_config()
        out_config = {
            **base_config,
            "filters": self.filters,
            "alpha": self.alpha,
            "filters": self.filters,
            "kernel_size": self.kernel_size,
            "strides": self.strides,
            "padding": self.padding,
            "kernel_initializer": self.kernel_initializer,
            "kernel_regularizer": self.kernel_regularizer,
            "kernel_constraint": self.kernel_constraint,
        }
        return out_config

In [11]:
def puigcerver_octconv():
    """
    Octave CNN by khinggan, architecture is same as puigcerver
    """

    alpha = 0.25
    input_data = Input(name="image", shape=(image_width, image_height, 1))
    labels = tf.keras.layers.Input(name="label", shape=(None,))
    high = input_data
    low = tf.keras.layers.AveragePooling2D(2)(input_data)

    high, low = OctConv2D(filters=16, alpha=alpha)([high, low])
    high = BatchNormalization()(high)
    low = BatchNormalization()(low)
    high = LeakyReLU(alpha=0.01)(high)
    low = LeakyReLU(alpha=0.01)(low)
    high = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="valid")(high)
    low = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="valid")(low)

    high, low = OctConv2D(filters=32, alpha=alpha)([high, low])
    high = BatchNormalization()(high)
    low = BatchNormalization()(low)
    high = LeakyReLU(alpha=0.01)(high)
    low = LeakyReLU(alpha=0.01)(low)
    high = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="valid")(high)
    low = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="valid")(low)

    high = Dropout(rate=0.2)(high)
    low = Dropout(rate=0.2)(low)
    high = Conv2D(filters=48, kernel_size=(3, 3), strides=(1, 1), padding="same")(high)
    low = Conv2D(filters=48, kernel_size=(3, 3), strides=(1, 1), padding="same")(low)
    high = BatchNormalization()(high)
    low = BatchNormalization()(low)
    high = LeakyReLU(alpha=0.01)(high)
    low = LeakyReLU(alpha=0.01)(low)
    high = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="valid")(high)
    low = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="valid")(low)

    high = Dropout(rate=0.2)(high)
    low = Dropout(rate=0.2)(low)
    high = Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding="same")(high)
    low = Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding="same")(low)
    high = BatchNormalization()(high)
    low = BatchNormalization()(low)
    high = LeakyReLU(alpha=0.01)(high)
    low = LeakyReLU(alpha=0.01)(low)

    high = Dropout(rate=0.2)(high)
    low = Dropout(rate=0.2)(low)
    high = Conv2D(filters=80, kernel_size=(3, 3), strides=(1, 1), padding="same")(high)
    low = Conv2D(filters=80, kernel_size=(3, 3), strides=(1, 1), padding="same")(low)
    high = BatchNormalization()(high)
    low = BatchNormalization()(low)
    high = LeakyReLU(alpha=0.01)(high)
    low = LeakyReLU(alpha=0.01)(low)

    x = _create_octconv_last_block([high, low], 80, alpha)

    shape = x.get_shape()
    blstm = Reshape((shape[1], shape[2] * shape[3]))(x)

    blstm = Bidirectional(LSTM(units=256, return_sequences=True, dropout=0.5))(blstm)
    blstm = Bidirectional(LSTM(units=256, return_sequences=True, dropout=0.5))(blstm)
    blstm = Bidirectional(LSTM(units=256, return_sequences=True, dropout=0.5))(blstm)
    blstm = Bidirectional(LSTM(units=256, return_sequences=True, dropout=0.5))(blstm)
    blstm = Bidirectional(LSTM(units=256, return_sequences=True, dropout=0.5))(blstm)

    blstm = Dropout(rate=0.5)(blstm)
    output_data = Dense(units=len(char_to_num.get_vocabulary()) + 2, activation="softmax", name="dense")(blstm)

    output = CTCLayer(name="ctc_loss")(labels, output_data)

    # Define the model.
    model = tf.keras.models.Model(
        inputs=[input_data, labels], outputs=output, name="handwriting_recognizer"
    )
    # Optimizer.
    opt = tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, clipnorm=1.0)
    # Compile the model and return.
    model.compile(optimizer=opt)
    return model


def _create_octconv_last_block(inputs, ch, alpha):
    high, low = inputs

    high, low = OctConv2D(filters=ch, alpha=alpha)([high, low])
    high = BatchNormalization()(high)
    high = Activation("relu")(high)

    low = BatchNormalization()(low)
    low = Activation("relu")(low)

    high_to_high = Conv2D(ch, 3, padding="same")(high)
    low_to_high = Conv2D(ch, 3, padding="same")(low)
    low_to_high = Lambda(lambda x: K.repeat_elements(K.repeat_elements(x, 2, axis=1), 2, axis=2))(low_to_high)

    x = Add()([high_to_high, low_to_high])
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

model = puigcerver_octconv()
model.summary()

Model: "handwriting_recognizer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 image (InputLayer)             [(None, 512, 64, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 average_pooling2d (AveragePool  (None, 256, 32, 1)  0           ['image[0][0]']                  
 ing2D)                                                                                           
                                                                                                  
 oct_conv2d (OctConv2D)         [(None, 512, 64, 12  288         ['image[0][0]',                  
                                ),                                'average_po

In [ ]:
# def build_model():
#     # Inputs to the model
#     input_img = keras.Input(shape=(image_width, image_height, 1), name="image")
#     labels = keras.layers.Input(name="label", shape=(None,))

#     # x = tf.transpose(input_img, perm=[0, 2, 1, 3])

#     cnn = keras.layers.Conv2D(filters=16, kernel_size=(3, 3), strides=(1, 1), padding="same")(input_img)
#     cnn = keras.layers.BatchNormalization()(cnn)
#     cnn = keras.layers.LeakyReLU(alpha=0.01)(cnn)
#     cnn = keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="valid")(cnn)

#     cnn = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), padding="same")(cnn)
#     cnn = keras.layers.BatchNormalization()(cnn)
#     cnn = keras.layers.LeakyReLU(alpha=0.01)(cnn)
#     cnn = keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="valid")(cnn)

#     cnn = keras.layers.Dropout(rate=0.2)(cnn)
#     cnn = keras.layers.Conv2D(filters=48, kernel_size=(3, 3), strides=(1, 1), padding="same")(cnn)
#     cnn = keras.layers.BatchNormalization()(cnn)
#     cnn = keras.layers.LeakyReLU(alpha=0.01)(cnn)
#     # cnn = keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="valid")(cnn)

#     cnn = keras.layers.Dropout(rate=0.2)(cnn)
#     cnn = keras.layers.Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding="same")(cnn)
#     cnn = keras.layers.BatchNormalization()(cnn)
#     cnn = keras.layers.LeakyReLU(alpha=0.01)(cnn)

#     cnn = keras.layers.Dropout(rate=0.2)(cnn)
#     cnn = keras.layers.Conv2D(filters=80, kernel_size=(3, 3), strides=(1, 1), padding="same")(cnn)
#     cnn = keras.layers.BatchNormalization()(cnn)
#     cnn = keras.layers.LeakyReLU(alpha=0.01)(cnn)



#     shape = cnn.get_shape()
#     blstm = keras.layers.Reshape((shape[1], shape[2] * shape[3]))(cnn)

#     # RNNs.
#     x = keras.layers.Bidirectional(
#         keras.layers.LSTM(128, return_sequences=True, dropout=0.5)
#     )(blstm)
#     x = keras.layers.Bidirectional(
#         keras.layers.LSTM(128, return_sequences=True, dropout=0.5)
#     )(x)
#     # x = keras.layers.Bidirectional(
#     #     keras.layers.LSTM(128, return_sequences=True, dropout=0.5)
#     # )(x)
#     # x = keras.layers.Bidirectional(
#     #     keras.layers.LSTM(128, return_sequences=True, dropout=0.5)
#     # )(x)

#     # +2 is to account for the two special tokens introduced by the CTC loss.
#     # The recommendation comes here: https://git.io/J0eXP.
#     x = keras.layers.Dense(
#         len(char_to_num.get_vocabulary()) + 2, activation="softmax", name="dense"
#     )(x)

#     # Add CTC layer for calculating CTC loss at each step.
#     output = CTCLayer(name="ctc_loss")(labels, x)

#     # Define the model.
#     model = keras.models.Model(
#         inputs=[input_img, labels], outputs=output, name="handwriting_recognizer"
#     )
#     # Optimizer.
#     opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, clipnorm=1.0)
#     # Compile the model and return.
#     model.compile(optimizer=opt)
#     return model

# model = build_model()
# model.summary()

In [12]:
# A utility function to decode the output of the network.
def decode_batch_predictions(pred):
    pred = tf.transpose(pred, perm=[1, 0, 2])
    pred = tf.nn.ctc_greedy_decoder( pred, [pred.shape[0]]*pred.shape[1], blank_index=112)
    # decode the output from ctc decoder
    batch_size = pred[0][0].shape[0]
    # create a list list of size batch_size
    decoded = [[] for _ in range(batch_size)]
    total_values = pred[0][0].values.shape[0]
    for i in range(total_values):
        idx = pred[0][0].indices[i][0]
        decoded[idx].append(pred[0][0].values[i])

    # decode decoded list to text
    decoded_text = [num_to_char(decoded[i]) for i in range(batch_size)]
    # convert to string
    decoded_text = [tf.strings.reduce_join(decoded_text[i]) for i in range(batch_size)]
    # convert to numpy
    decoded_text = [decoded_text[i].numpy().decode("utf-8") for i in range(batch_size)]
    return decoded_text

# A utility function to decode the output of the network.
def decode_batch_predictions_beam(pred):
    pred = tf.transpose(pred, perm=[1, 0, 2])
    pred = tf.nn.ctc_beam_search_decoder( pred, [pred.shape[0]]*pred.shape[1])
    # decode the output from ctc decoder
    batch_size = pred[0][0].shape[0]
    # create a list list of size batch_size
    decoded = [[] for _ in range(batch_size)]
    total_values = pred[0][0].values.shape[0]
    for i in range(total_values):
        idx = pred[0][0].indices[i][0]
        decoded[idx].append(pred[0][0].values[i])

    # decode decoded list to text
    decoded_text = [num_to_char(decoded[i]) for i in range(batch_size)]
    # convert to string
    decoded_text = [tf.strings.reduce_join(decoded_text[i]) for i in range(batch_size)]
    # convert to numpy
    decoded_text = [decoded_text[i].numpy().decode("utf-8") for i in range(batch_size)]
    return decoded_text

figures_list = []

class PlotPredictions(tf.keras.callbacks.Callback):

    def __init__(self, pred_model, frequency=1):
        self.frequency = frequency
        self.prediction_model = pred_model
        super(PlotPredictions, self).__init__()

        batch = validation_ds.take(1)
        self.batch_images = list(batch.as_numpy_iterator())[0]["image"]
        self.batch_labels = list(batch.as_numpy_iterator())[0]["label"]

    def plot_predictions(self, epoch):

        prediction_model = keras.models.Model(
            self.model.get_layer(name="image").input, 
            self.model.get_layer(name="dense").output
        )
        
        preds = self.prediction_model.predict(self.batch_images, verbose=0)
        pred_texts = decode_batch_predictions(preds)

        fig , ax = plt.subplots(4, 4, figsize=(15, 5))
        fig.suptitle('Epoch: '+str(epoch), weight='bold', size=14)

        for i in range(16):

            img = self.batch_images[i]
            img = tf.image.flip_left_right(img)
            img = tf.transpose(img, perm=[1, 0, 2])
            img = (img * 255.0).numpy().clip(0, 255).astype(np.uint8)
            img = img[:, :, 0]
            
            title = f"{pred_texts[i]}"
            ax[i // 4, i % 4].imshow(img, cmap="gray")
            ax[i // 4, i % 4].set_title(title)
            ax[i // 4, i % 4].axis("off")
        
        plt.show()        
        figures_list.append(fig)

    def on_epoch_end(self, epoch, logs=None):
        if epoch % self.frequency == 0:
            self.plot_predictions(epoch)



class EditDistanceCallback(keras.callbacks.Callback):
    def __init__(self, pred_model):
        super().__init__()
        self.prediction_model = pred_model

    def on_epoch_end(self, epoch, logs=None):
        # edit_distances = []
        # totol_dist = 0
        inf_dict = {}
        inf_dict_g = {}

        for item in validation_ds.as_numpy_iterator():
            labels = item["label"]
            predictions = self.prediction_model.predict(item["image"], verbose=0)
            
            pred_texts = decode_batch_predictions(predictions)
            pred_texts_g = decode_batch_predictions_beam(predictions)
            
            res = tf.gather(labels, tf.where(tf.math.not_equal(labels, 120)))
            true_texts = tf.strings.reduce_join(num_to_char(res)).numpy().decode("utf-8")
            for pred_text, pred_text_g, label in zip(pred_texts, pred_texts_g, labels):
                
                res = tf.gather(label, tf.where(tf.math.not_equal(label, 120)))
                res = tf.strings.reduce_join(num_to_char(res)).numpy().decode("utf-8")
                
                dist = editdistance.eval(pred_text, res)
                dist_g = editdistance.eval(pred_text_g, res)
                idx = round(len(res) / 5)
                
                if idx not in inf_dict:
                    inf_dict[idx] = {"total": 0, "num_word_ok": 0, "char_error_rate": 0, "total_char": 0}
                    inf_dict_g[idx] = {"total": 0, "num_word_ok": 0, "char_error_rate": 0, "total_char": 0}
                
                inf_dict[idx]["total"] += 1
                inf_dict[idx]["num_word_ok"] += 1 if pred_text == res else 0
                inf_dict[idx]["char_error_rate"] += dist
                inf_dict[idx]["total_char"] += len(res)

                inf_dict_g[idx]["total"] += 1
                inf_dict_g[idx]["num_word_ok"] += 1 if pred_text_g == res else 0
                inf_dict_g[idx]["char_error_rate"] += dist_g
                inf_dict_g[idx]["total_char"] += len(res)


        print("Greedy")        
        ordered_key = sorted(inf_dict.keys())
        for key in ordered_key:
            value = inf_dict[key]
            print(f"Length: \t {key*5}-{(key+1)*5} \t| Total: {value['total']} \t| CharErrorRate: {value['char_error_rate']/value['total_char']} \t| WordAccuracy: {value['num_word_ok']/value['total']}")

        print("Beam")
        ordered_key = sorted(inf_dict_g.keys())
        for key in ordered_key:
            value = inf_dict_g[key]
            print(f"Length: \t {key*5}-{(key+1)*5} \t| Total: {value['total']} \t| CharErrorRate: {value['char_error_rate']/value['total_char']} \t| WordAccuracy: {value['num_word_ok']/value['total']}")
 

In [ ]:
model_path = "drive/MyDrive/Dataset/checkpoint_puigcerver_octconv"

model = keras.models.load_model(model_path)

In [ ]:
prediction_model = tf.keras.models.Model(
    model.get_layer(name="image").input, model.get_layer(name="dense").output
)

# file_path = "./checkpoint_puigcerver_octconv"
file_path = "drive/MyDrive/Dataset/checkpoint_puigcerver_octconv3"
    
checkpoint = ModelCheckpoint(filepath=file_path, 
                            monitor='val_loss', 
                            verbose=1, 
                            save_best_only=True, 
                            mode='min')

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', min_delta=1e-8, factor=0.1, patience=5, cooldown=0)

# Train the model.
history = model.fit(
    train_ds,
    validation_data=validation_ds,
    epochs=200,
    callbacks=[reduce_lr, checkpoint])

Epoch 1/200
306/306 [==============================] - ETA: 0s - loss: 133.1949
Epoch 1: val_loss improved from inf to 171.43259, saving model to drive/MyDrive/Dataset/checkpoint_puigcerver_octconv3


306/306 [==============================] - 138s 323ms/step - loss: 133.1949 - val_loss: 171.4326 - lr: 0.0010
Epoch 2/200
306/306 [==============================] - ETA: 0s - loss: 108.7816
Epoch 2: val_loss improved from 171.43259 to 109.23646, saving model to drive/MyDrive/Dataset/checkpoint_puigcerver_octconv3


306/306 [==============================] - 92s 301ms/step - loss: 108.7816 - val_loss: 109.2365 - lr: 0.0010
Epoch 3/200
306/306 [==============================] - ETA: 0s - loss: 92.2013
Epoch 3: val_loss improved from 109.23646 to 85.02041, saving model to drive/MyDrive/Dataset/checkpoint_puigcerver_octconv3


306/306 [==============================] - 93s 306ms/step - loss: 92.2013 - val_loss: 85.0204 - lr: 0.0010
Epoch 4/200
306/306 [==============================] - ETA: 0s - loss: 81.8120
Epoch 4: val_loss improved from 85.02041 to 75.20926, saving model to drive/MyDrive/Dataset/checkpoint_puigcerver_octconv3


306/306 [==============================] - 92s 303ms/step - loss: 81.8120 - val_loss: 75.2093 - lr: 0.0010
Epoch 5/200
306/306 [==============================] - ETA: 0s - loss: 75.0609
Epoch 5: val_loss improved from 75.20926 to 72.38008, saving model to drive/MyDrive/Dataset/checkpoint_puigcerver_octconv3


...

306/306 [==============================] - 92s 303ms/step - loss: 32.3567 - val_loss: 29.7778 - lr: 0.0010
Epoch 22/200
306/306 [==============================] - ETA: 0s - loss: 31.2316
Epoch 22: val_loss improved from 29.77783 to 29.56514, saving model to drive/MyDrive/Dataset/checkpoint_puigcerver_octconv3


306/306 [==============================] - 92s 300ms/step - loss: 31.2316 - val_loss: 29.5651 - lr: 0.0010
Epoch 23/200
306/306 [==============================] - ETA: 0s - loss: 30.3253
Epoch 23: val_loss improved from 29.56514 to 28.85041, saving model to drive/MyDrive/Dataset/checkpoint_puigcerver_octconv3


306/306 [==============================] - 92s 301ms/step - loss: 30.3253 - val_loss: 28.8504 - lr: 0.0010
Epoch 24/200
306/306 [==============================] - ETA: 0s - loss: 29.5747
Epoch 24: val_loss improved from 28.85041 to 26.73828, saving model to drive/MyDrive/Dataset/checkpoint_puigcerver_octconv3


306/306 [==============================] - 92s 301ms/step - loss: 29.5747 - val_loss: 26.7383 - lr: 0.0010
Epoch 25/200
306/306 [==============================] - ETA: 0s - loss: 28.6756
Epoch 25: val_loss improved from 26.73828 to 25.85981, saving model to drive/MyDrive/Dataset/checkpoint_puigcerver_octconv3


306/306 [==============================] - 92s 301ms/step - loss: 28.6756 - val_loss: 25.8598 - lr: 0.0010
Epoch 26/200
306/306 [==============================] - ETA: 0s - loss: 27.7346
Epoch 26: val_loss did not improve from 25.85981
306/306 [==============================] - 26s 86ms/step - loss: 27.7346 - val_loss: 26.4050 - lr: 0.0010
Epoch 27/200
120/306 [==========>...................] - ETA: 15s - loss: 27.5159